In [2]:
# 安装LightGBM
!pip install lightgbm --install-option=--gpu

D:\anoconda\lib\site-packages\pip\_internal\commands\install.py:230: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)


In [1]:
# 导入
import lightgbm as lgb
import pandas as pd
import json
from sklearn import datasets

In [2]:
# 导入鸢尾花数据集
iris = datasets.load_iris()
iris.data.shape

#构建数据集
from sklearn.model_selection import train_test_split as TTS
#将iris划分为训练集和测试机
train_data_all,test_data, train_y_all, test_y = TTS(iris.data, iris.target, test_size=0.2, random_state=42, shuffle=True)
train_data, val_data, train_y, val_y = TTS(train_data_all, train_y_all, test_size=0.2, random_state=42, shuffle=True)
train_data.shape

#构建LGB的训练集
lgb_train = lgb.Dataset(train_data, train_y)
lgb_val = lgb.Dataset(val_data, val_y)

#设置模型参数
params = {
    "objective":"multiclass",
    "num_classes":3,
    "verbosity":-1
}
# 训练模型
booster = lgb.train(params, train_set=lgb_train, valid_sets=lgb_val, num_boost_round=12)

#使用训练的模型对iris进行预测
from sklearn.metrics import accuracy_score
import numpy as np

train_preds = np.argmax(booster.predict(train_data), axis=1)
test_preds = np.argmax(booster.predict(test_data), axis=1)
#测试结果
print("Train Accuracy Score: %.2f"%accuracy_score(train_y, train_preds))
print("Test Accuracy Score: %.2f"%accuracy_score(test_y, test_preds))

[1]	valid_0's multi_logloss: 0.998257
[2]	valid_0's multi_logloss: 0.87888
[3]	valid_0's multi_logloss: 0.802809
[4]	valid_0's multi_logloss: 0.723778
[5]	valid_0's multi_logloss: 0.65558
[6]	valid_0's multi_logloss: 0.60953
[7]	valid_0's multi_logloss: 0.553635
[8]	valid_0's multi_logloss: 0.499817
[9]	valid_0's multi_logloss: 0.471947
[10]	valid_0's multi_logloss: 0.438747
[11]	valid_0's multi_logloss: 0.404308
[12]	valid_0's multi_logloss: 0.387417
Train Accuracy Score: 0.96
Test Accuracy Score: 0.97


In [4]:
#模型保存
import pickle
pickle.dump(booster,open('model.pickle','wb'))

In [22]:
#使用txt进行保存
booster.save_model('model.txt')

In [24]:
#加载模型，并进行预测
model_pickle = pickle.load(open('model.pickle','rb'))
y_pred_pickle = np.argmax(model_pickle.predict(test_data), axis=1)
print("y_pred_pickle:",y_pred_pickle)
# txt读取
model_txt = lgb.Booster(model_file='model.txt')
y_pred_txt = np.argmax(model_txt.predict(test_data), axis=1)
print("y_pred_txt:",y_pred_txt)

y_pred_pickle: [1 0 2 1 2 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
y_pred_txt: [1 0 2 1 2 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
